#### Getting Correct Release and Publication Dates

This notebook is designed to adjust the minutes_info.xlsx file by adding the correct reference and publication dates for each Copom minute.
 * It connects to the Brazilian Central Bank API to get the relevant dates.
 * The new information, is saved back to the minutes_info.xlsx file.
---
 * Note: The analysis will begins with minute #85 (2003-06-27). This starting point is determined by the accessibility of historical data for the Brazilian interest rate term structure (ETTJ) from the sources consulted for this project, which is crucial for the econometric stage of this research.
---

In [1]:
import requests
import pandas as pd

In [ ]:
FOLDER_RAW = "./data/raw"

In [ ]:
# Extracting and saving info about the Copom Minutes from the Brazilian Central Bank API.
url = "https://www.bcb.gov.br/api/servico/sitebcb/copom/atas?quantidade=1000"

response = requests.get(url)

In [ ]:
df_minutes_date = pd.DataFrame(response.json()["conteudo"])
df_minutes_date["dataReferencia"] = pd.to_datetime(df_minutes_date["dataReferencia"], format="%Y-%m-%d").dt.date
df_minutes_date["dataPublicacao"] = pd.to_datetime(df_minutes_date["dataPublicacao"], format="%Y-%m-%d").dt.date
df_minutes_date.set_index("nroReuniao", inplace=True)

df_minutes_date = df_minutes_date.tail(len(df_minutes_date) - 1)
df_minutes_date = df_minutes_date.head(len(df_minutes_date) - 21)

In [ ]:
df_info = pd.read_excel(f"{FOLDER_RAW}/minutes_info.xlsx")
df_info["DataReferencia"] = pd.to_datetime(df_info["DataReferencia"]).dt.date

In [ ]:
df_info["nroReuniao"] = ""

for index, row in df_info.iterrows():
    try:
        nroReuniao = row["Titulo"][0:3]
        nroReuniao = int(nroReuniao)
    except:
        nroReuniao = row["Titulo"][0:2]
        nroReuniao = int(nroReuniao)

    df_info.at[index, "nroReuniao"] = nroReuniao

In [ ]:
df_info["DataReferencia"] = df_info["nroReuniao"].map(df_minutes_date["dataReferencia"])
df_info["DataPublicacao"] = df_info["nroReuniao"].map(df_minutes_date["dataPublicacao"])	

In [ ]:
df_info.as_excel(f"{FOLDER_RAW}/minutes_info.xlsx", index=False)